# This notebook will create the df_timit and save it as a csv for later use. 
    - We need to step through all the directories in the timit data store and create necessary rows
    - Also need to augment the data with some additional info
    
## Dialect information
     dr1:  New England
     dr2:  Northern
     dr3:  North Midland
     dr4:  South Midland
     dr5:  Southern
     dr6:  New York City
     dr7:  Western
     dr8:  Army Brat (moved around)


In [11]:
import pandas as pd
import numpy as np
import os
import csv
import pdb
from tqdm import tqdm
import random

#increase some pandas vars
pd.set_option('max_colwidth', 100)
sys_name = os.uname()[1]

def data_path():    
    return "/home/logan/Downloads/CI_simmed_processed/CI_simed"
        

def append_file(root, file, hive=False, add_slash=False):
    if hive:
        mod_root = "E:\\SynologyDrive\\" + root[root.find('Research'):]
        if add_slash:
            output = mod_root  + '\\' + file
        else:
            output = mod_root  + file
        output =  output.replace('/', '\\')
    else:
        if add_slash:
            output = root + '/' + file
        else:
            output = root + file
    return output

def cut_speaker_id(path):
    return path[path.rfind('/') + 1:]

In [12]:
# arpabet to ipa conversion
arpa_key = ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'axr', 'ay', 'eh', 'er', 'ey', 'ih', 'ix', 'iy', \
                'ow', 'oy', 'uh', 'uw', 'ux' , 'b', 'ch', 'd', 'dh', 'dx', 'el', 'em', 'en', 'f', \
                'g', 'h', 'hh', 'jh', 'k', 'l', 'm', 'n', 'ng', 'nx', 'p', 'q', 'r', 's', 'sh', \
                't', 'th', 'v', 'w', 'wh', 'y', 'z', 'zh', 'ax-h', 'bcl', 'dcl', 'eng', 'gcl', 'hv', \
                'kcl', 'pcl', 'tcl', 'pau', 'epi', 'h#']
ipa_key = ['ɑ', 'æ', 'ʌ', 'ɔ', 'aʊ', 'ə', 'ɚ', 'aɪ', 'ɛ', 'ɝ', 'eɪ', 'ɪ', 'ɨ', 'i', 'oʊ', 'ɔɪ', \
                'ʊ', 'u', 'ʉ', 'b', 'tʃ', 'd', 'ð', 'ɾ', 'l̩', 'm̩', 'n̩', 'f', 'ɡ', 'h', 'h', 'dʒ', 'k', \
                'l', 'm', 'n', 'ŋ', 'ɾ̃', 'p', 'ʔ', 'ɹ', 's', 'ʃ', 't', 'θ', 'v', 'w', 'ʍ', 'j', 'z', \
                'ʒ', 'ə̥', 'b̚', 'd̚', 'ŋ̍', 'ɡ̚', 'ɦ', 'k̚', 'p̚', 't̚', 'N/A', 'N/A', 'N/A']

ipa_conversion = dict(zip(arpa_key, ipa_key))
def convert_to_ipa(arpa_key):
    output = []
    for key in arpa_key:
        output.append(ipa_conversion[key])
    return output

arpa_conversion = dict(zip(ipa_key, arpa_key))
def convert_from_ipa(ipa_key):
    output = []
    for key in ipa_key:
        output.append(arpa_conversion.get(key, 'N/A'))
    return output

def join_word_phoneme(df_wrd, df_phn, audio_file):
    new_df = df_wrd.merge(df_phn, 'outer', on=('sample_id', 'speaker_id'), \
                          suffixes=('_word', '_phoneme'))
    new_df = new_df[(new_df.start_phoneme >= new_df.start_word) &
                    (new_df.end_phoneme <= new_df.end_word)]
    try:
        new_df['ipa'] = convert_to_ipa(new_df['arpabet'])
        new_df['filepath'] = audio_file
    except:
        pdb.set_trace()
    if audio_file[audio_file.rfind('/') - 5] == 'M':
        new_df['sex'] = 'm'
    else:
        new_df['sex'] = 'f'
    return new_df

def ipa_join_word_phoneme(df_wrd, df_phn, audio_file):
    new_df = df_wrd.merge(df_phn, 'outer', on=('sample_id', 'speaker_id'), \
                          suffixes=('_word', '_phoneme'))
    new_df = new_df[(new_df.start_phoneme >= new_df.start_word) &
                    (new_df.end_phoneme <= new_df.end_word)]
    try:
        new_df['arpabet'] = convert_from_ipa(new_df['ipa'])
        new_df['filepath'] = audio_file
    except:
        pdb.set_trace()
    if audio_file[audio_file.rfind('/') - 5] == 'M':
        new_df['sex'] = 'm'
    else:
        new_df['sex'] = 'f'
    return new_df

In [15]:
#step through all directories in data path, consume all files with same name (different extensions)
#to populate a single row
tmp = 0
df_train = pd.DataFrame(columns=['start_word', 'end_word', 'word', 'sample_id', 'speaker_id', 'start_phoneme', 
        'end_phoneme', 'sex', 'arpabet', 'ipa', 'filepath'])
for root, dirs, files in list(os.walk(data_path())):     #training data
    for file in tqdm(files, position=0, leave=True):
        try:
            if 'wav' in file.lower():
                last_under = file.rfind('_')
                sample_id = file[last_under + 1:-4]
                speaker_id = file[last_under - 5:last_under]
                #get phoneme infor
                phn_file = append_file(root, file[:-4]+'.PHN', add_slash=True)
                df_phn = pd.read_csv(phn_file, delimiter=' ', names=['start', 'end', 'ipa'])
                df_phn['sample_id'] = sample_id
                df_phn['speaker_id'] = speaker_id

                #get word info
                wrd_file = append_file(root, file[:-4]+'.WRD', add_slash=True)
                df_wrd = pd.read_csv(wrd_file, delimiter=' ', names=['start', 'end', 'word'])
                df_wrd['sample_id'] = sample_id
                df_wrd['speaker_id'] = speaker_id
                df_train = df_train.append(ipa_join_word_phoneme(df_wrd, df_phn, 
                                        append_file(root, file, add_slash=True)), ignore_index=True)
        except:
            pass

#add phoneme indices for bigram analysis
df_train['index_phoneme'] = -1
grouped_df = df_train.groupby(['speaker_id', 'sample_id', 'start_word'])
for key, item in grouped_df:
    new_indices = list(range(len(item.index_phoneme)))
    df_train.loc[item.index, 'index_phoneme'] = new_indices
    
#filter out unknown ipa values (epi, silence)
all_ph = ['ʃ', 'ɨ', 'ɦ', 'ɛ', 'd̚', 'dʒ', 'ɪ', 'd', 'ʌ', 'k̚', 'k', 's', 'ʉ',
       'ʔ', 'n̩', 'ɡ̚', 'ɡ', 'ɹ', 'w', 'ɔ', 'ɾ', 'ɚ', 'l', 'j', 'ʊ', 'n',
       'æ', 'm', 'ɔɪ', 'ə', 'ð', 't̚', 'i', 'v', 'f', 't', 'p̚', 'oʊ',
       'h', 'tʃ', 'b̚', 'b', 'ɑ', 'm̩', 'ŋ', 'aɪ', 'θ', 'ə̥', 'eɪ',
       'p', 'aʊ', 'ɝ', 'ɾ̃', 'z', 'l̩', 'u', 'ʒ', 'ŋ̍']

df_train = df_train[df_train.ipa.isin(all_ph)]
df_train.count()[0]

#save dataframe
df_train.to_csv('../../data/loc_master.csv', index=False)
print('Done')

100%|██████████| 263/263 [00:00<00:00, 1472766.29it/s]


Done
